In [ ]:
path_folder = '../resource/stock_market_data'

In [1]:
import duckdb

# Create a DuckDB connection
conn = duckdb.connect()
directory_path = 'C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\csv\\*.csv'

# Create a table from all CSV files
conn.execute(f"CREATE TABLE my_table AS SELECT * FROM read_csv('{directory_path}', union_by_name=True)")

# Execute a query
result = conn.execute("SELECT * FROM my_table limit 100").fetchdf()

# Print the result
print(result)


         Date        Low       Open      Volume       High      Close  \
0  1999-11-18  28.612303  32.546494  62546380.0  35.765381  31.473534   
1  1999-11-19  28.478184  30.713518  15234146.0  30.758226  28.880545   
2  1999-11-22  28.657009  29.551144   6577870.0  31.473534  31.473534   
3  1999-11-23  28.612303  30.400572   5975611.0  31.205294  28.612303   
4  1999-11-24  28.612303  28.701717   4843231.0  29.998213  29.372318   
..        ...        ...        ...         ...        ...        ...   
95 2000-04-05  64.467094  66.344780   2109163.0  71.441345  69.340126   
96 2000-04-06  69.429543  70.368385   1375073.0  75.107300  75.107300   
97 2000-04-07  76.582619  76.895561   2975224.0  89.413445  87.267525   
98 2000-04-10  75.375534  87.267525   2956071.0  87.267525  76.627327   
99 2000-04-11  71.217812  72.246063   1896107.0  75.017883  72.201363   

    Adjusted Close  
0        27.066582  
1        24.836662  
2        27.066582  
3        24.605980  
4        25.259573

In [2]:
import duckdb
import glob

# Specify the path to your CSV files
path_to_files = 'C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\csv\\*.csv'

# Use glob to find all CSV files in the specified directory
files = glob.glob(path_to_files)

ModuleNotFoundError: No module named 'duckdb'

In [ ]:
import os
file_name = os.path.basename("C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\csv\\A.csv")

In [ ]:
file_name = []
for file_path in files:
    file_base = os.path.basename(file_path)
    file_name.append(file_base)

In [ ]:
#put 100 companies files for memory issue
files = files[:100]
file_name = file_name[:100]

In [ ]:
conn = duckdb.connect()
queries = []
for path_file, file in zip(files, file_name):
    queries.append(f"SELECT replace('{file}','.csv','') AS company_id, *  FROM read_csv('{path_file}', union_by_name=True)")
    
final_query = " UNION ALL ".join(queries)
conn.execute(f"SET max_expression_depth TO 2000")
conn.execute(f"CREATE TABLE forbes2000_from_csv AS {final_query}")



In [ ]:
results = conn.execute("SELECT * from forbes2000_from_csv limit 100").fetchdf()
results.head(10)

In [ ]:
conn.close()

In [ ]:
# Create a DuckDB connection


# Initialize an empty list to hold queries
queries = []

# Construct the query for each file to include the filename
for file in files:
    queries.append(f"SELECT *, '{file}' AS filename FROM read_csv_auto('{file}')")

# Combine all queries with UNION ALL
final_query = " UNION ALL ".join(queries)

# Execute the combined query
result_df = conn.execute(final_query).fetchdf()


In [4]:
test = 'C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\json\\A.json'

In [39]:
# Replace 'your_file.json' with the path to your JSON file
result = conn.execute(
    """
    SELECT 
        result[0].meta.currency AS currency,
        result[0].meta.symbol AS symbol,
        result[0].meta.exchangeName AS exchange_name,
        result[0].meta.instrumentType AS instrument_type,
        result[0].meta.firstTradeDate AS first_trade_date,
        result[0].meta.regularMarketTime AS regular_market_time,
        result[0].meta.regularMarketPrice AS regular_market_price,
        result[0].meta.chartPreviousClose AS chart_previous_close
    FROM (
        SELECT chart.result FROM read_json('C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\json\\AA-P.json') AS chart
    ) AS nested_json
    """)

In [50]:
import duckdb
print(duckdb.__version__)

1.1.1


In [40]:
result.fetchdf()

,currency,symbol,exchange_name,instrument_type,first_trade_date,regular_market_time,regular_market_price,chart_previous_close
0,None,None,None,None,NaN,NaN,NaN,NaN


In [71]:
import json
import duckdb

# Load and print the contents of the JSON file for verification
with open('C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\json\\AA-P.json') as f:
    data = json.load(f)
    print("Loaded JSON Data:")
    print(json.dumps(data, indent=2))  # Check JSON structure

# Connect to a DuckDB database (or create one in-memory)
con = duckdb.connect(database=':memory:')  # or 'your_database.db'

# Create a table from the JSON data directly
con.execute("CREATE TABLE chart AS SELECT * FROM read_json('C:\\Users\\diasn\\Documents\\medium\\duckdb_postgre\\resources\\stock_market_data\\forbes2000\\json\\AA-P.json')")

# Now, let's check the structure of the table
df = con.execute("SELECT * FROM chart").fetchdf()
print("Raw Data from Table:")
print(df)

# Attempt to extract specific fields with the correct path
nested_query = """
    SELECT 
        result[0].meta.currency AS currency,
        result[0].meta.symbol AS symbol,
        result[0].meta.exchangeName AS exchange_name,
        result[0].meta.instrumentType AS instrument_type,
        result[0].meta.firstTradeDate AS first_trade_date,
        result[0].meta.regularMarketTime AS regular_market_time
    FROM (SELECT chart.chart.result AS result FROM chart)
"""

# Execute the nested query
nested_df = con.execute(nested_query).fetchdf()
print("Nested Query Results:")
print(nested_df)

# Close the connection
con.close()


Loaded JSON Data:
{
  "chart": {
    "result": [
      {
        "meta": {
          "currency": null,
          "symbol": "AA-P",
          "exchangeName": "YHD",
          "instrumentType": "MUTUALFUND",
          "firstTradeDate": null,
          "regularMarketTime": 1561759658,
          "gmtoffset": -18000,
          "timezone": "EST",
          "exchangeTimezoneName": "America/New_York",
          "priceHint": 2,
          "currentTradingPeriod": {
            "pre": {
              "timezone": "EST",
              "start": 1670835600,
              "end": 1670855400,
              "gmtoffset": -18000
            },
            "regular": {
              "timezone": "EST",
              "start": 1670855400,
              "end": 1670878800,
              "gmtoffset": -18000
            },
            "post": {
              "timezone": "EST",
              "start": 1670878800,
              "end": 1670893200,
              "gmtoffset": -18000
            }
          },
          "

In [72]:
nested_df

,currency,symbol,exchange_name,instrument_type,first_trade_date,regular_market_time
0,None,None,None,None,None,NaN


In [57]:
# Close the connection
con.close()